# 2.3.2 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Best Ergebnisse (text_prompts, img_prompts): A2, E1, E2, [B3,C3,D3]

### Setup

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()
runner = Runner()

In [2]:
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F11-01-TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print("Seitenzahl:", len(pdf_prompts))

Seitenzahl: 10


### A) OCR komplettes PDF

In [3]:
instruction = 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
instruction_prompt = parser.text2prompts([instruction])

In [4]:
# A1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Komplettes PDF auf einmal einlesen. Ziel: Konsistentes Layout.
pdf_text = ava.request([*pdf_prompts, *instruction_prompt])
pprint(pdf_text)

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\section*{Schriftlicher Teil (Teil B)}
\textbf{Bebauungsplan "Gewerbegebiet Himmelreich"}\\
Stadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis

\subsection*{}
Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.\\
Lageplan M 1:500

\subsection*{}
Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:
\begin{itemize}
    \item Baugesetzbuch (BauGB) in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    \item Baunutzungsverordnung (BauNVO) in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    \item Planzeichenverordnung 1990 (PlanzV 90) in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl. I. S. 58).
    \item Landesbauordnung (LBO) in der Fassung der Bekanntmachung vom 08.08.1995 (GBl. S. 617).
\end{itemize}

\subsection*{Bisherige Festsetzungen:}
Mit in Kraf

In [5]:
# A2) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten. Ziel: Fokus maximieren und Context-Window klein halten + Seitenzahlen beleiben erhalten.
prompt_chain = list(map(lambda prompt: ava.lambdaRequest([prompt, *instruction_prompt]), pdf_prompts))
msgf11pdf = await asyncio.gather(*prompt_chain)
pprint(msgf11pdf)
%store msgf11pdf

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\begin{flushright}
\includegraphics[width=0.2\textwidth]{logo.png} \\
19.06.2000 \\
9833
\end{flushright}

\begin{center}
\textbf{SCHRIFTLICHER TEIL (Teil B)} \\
\textbf{BEBAUUNGSPLAN "GEWERBEGEBIET HIMMELREICH"} \\
\textbf{STADT LAICHINGEN, GEMARKUNG FELDSTETTEN, ALB-DONAU-KREIS}
\end{center}

Der Geltungsbereich wird durch das Planzeichen im Lageplan begrenzt.

Lageplan M 1: 500

Für die planungsrechtlichen bzw. bauordnungsrechtlichen Festsetzungen gelten:

\begin{itemize}
    \item \textbf{Baugesetzbuch (BauGB)} \\
    in der Fassung der Bekanntmachung vom 27.08.1997 (BGBl. I. S. 2141).
    
    \item \textbf{Baunutzungsverordnung (BauNVO)} \\
    in der Fassung der Bekanntmachung vom 23.01.1990 (BGBl. S. 132), zuletzt geändert am 22.04.1993 (BGBl. I. S. 466).
    
    \item \textbf{Planzeichenverordnung 1990 (PlanZV 90)} \\
    in der Fassung der Bekanntmachung vom 18.12.1990 (BGBl

### B) Art der baulichen Nutzung

1. text_prompts
2. img_prompts

In [6]:
%store -r msgbaunvo_art msgf11pdf
context_art = parser.text2prompts([msgbaunvo_art])

In [7]:
# B1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".  Achte auf die Stuktur.'

def run():    
    return ava.similaritySearchWithContext(instruction, msgf11pdf, context_art)
runner.consistency_check(run)

['### Art der baulichen Nutzung (§§ 1 - 15 BauNVO)

#### 1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)

##### 1.1.1 Zulässig sind / zulässig ist:
- **1.1.1.1.1** Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
- **1.1.1.1.2** Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.

##### 1.1.2 Nicht zulässig sind / nicht zulässig ist:
- **1.1.1.2.1** Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
- **1.1.1.2.2** Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gemäß § 1 (5) 

[['### Art der baulichen Nutzung (§§ 1 - 15 BauNVO)\n\n#### 1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)\n\n##### 1.1.1 Zulässig sind / zulässig ist:\n- **1.1.1.1.1** Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.\n- **1.1.1.1.2** Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.\n\n##### 1.1.2 Nicht zulässig sind / nicht zulässig ist:\n- **1.1.1.2.1** Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.\n- **1.1.1.2.2** Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten sind gem

In [8]:
# B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'

async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
await runner.async_consistency_check(run)

['### Art der baulichen Nutzung: 
1. **Planungsrechtliche Festsetzungen (§ 9 (1) und (2) BauGB und BauNVO)**
   - **1.1 Art der baulichen Nutzung (§§ 1 - 15 BauNVO)**
     - **1.1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)**
       - **1.1.1.1 Zulässig sind / zulässig ist:**
         - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
         - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
       - **1.1.1.2 Nicht zulässig sind / nicht zulässig ist:**
         - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
         - E

[['### Art der baulichen Nutzung: \n1. **Planungsrechtliche Festsetzungen (§ 9 (1) und (2) BauGB und BauNVO)**\n   - **1.1 Art der baulichen Nutzung (§§ 1 - 15 BauNVO)**\n     - **1.1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)**\n       - **1.1.1.1 Zulässig sind / zulässig ist:**\n         - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.\n         - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.\n       - **1.1.1.2 Nicht zulässig sind / nicht zulässig ist:**\n         - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.\n  

### C) Maß der baulichen Nutzung

1. text_prompts
2. img_prompts

In [9]:
%store -r msgbaunvo_maß msgf11pdf
context_maß = parser.text2prompts([msgbaunvo_maß])

In [10]:
# C1) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".  Achte auf die Stuktur.'

def run():
    return ava.similaritySearchWithContext(instruction, msgf11pdf, context_maß)
runner.consistency_check(run)

['Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":

---

### 1.2 Maß der baulichen Nutzung (§ 16 - 21a BauNVO)

#### 1.2.1 Grundflächenzahl (§ 19 BauNVO)
- siehe Einträge im Lageplan -

#### 1.2.2 Geschossflächenzahl (§ 20 BauNVO)
- siehe Einträge im Lageplan -

#### 1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)
- siehe Einträge im Lageplan -
- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.

---

Diese Informationen umfassen die relevanten Paragraphen der Baunutzungsverordnung (BauNVO) und geben an, dass spezifische Details im Lageplan zu finden sind. Die Höhe der baulichen Anlagen wird zudem genauer definiert.', [(Document(page_content='\\subsection*{1.1.5 Randsortimente}
Innenstadtrelevante, branchentypische Randsortimente sind in den zulässigen Verkaufsflächen gemäß 1.1.1.1.1 und 1.1.2.1.1 bis 

[['Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Gewerbegebiet Himmelreich":\n\n---\n\n### 1.2 Maß der baulichen Nutzung (§ 16 - 21a BauNVO)\n\n#### 1.2.1 Grundflächenzahl (§ 19 BauNVO)\n- siehe Einträge im Lageplan -\n\n#### 1.2.2 Geschossflächenzahl (§ 20 BauNVO)\n- siehe Einträge im Lageplan -\n\n#### 1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)\n- siehe Einträge im Lageplan -\n- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.\n\n---\n\nDiese Informationen umfassen die relevanten Paragraphen der Baunutzungsverordnung (BauNVO) und geben an, dass spezifische Details im Lageplan zu finden sind. Die Höhe der baulichen Anlagen wird zudem genauer definiert.',
  [(Document(page_content='\\subsection*{1.1.5 Randsortimente}\nInnenstadtrelevante, branchentypische Randsortimente sind in den zulässigen Verkaufsflächen gemäß 1.1.1.

In [11]:
# C3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF

instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'
async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_maß], img_type="pdf")
await runner.async_consistency_check(run)

['### Maß der baulichen Nutzung: 
1.2 Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)
1.2.1 Grundflächenzahl (§ 19 BauNVO)
- siehe Einschriebe im Lageplan

1.2.2 Geschossflächenzahl (§ 20 BauNVO)
- siehe Einschriebe im Lageplan

1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)
- siehe Einschriebe im Lageplan
- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.

### Referenzen: Seite 3']
#############################################
['### Maß der baulichen Nutzung: 
1. **Grundflächenzahl (§ 19 BauNVO)**
   - Siehe Einschreibung im Lageplan.

2. **Geschossflächenzahl (§ 20 BauNVO)**
   - Siehe Einschreibung im Lageplan.

3. **Höhe der baulichen Anlagen (§ 18 BauNVO)**
   - Siehe Einschreibung im Lageplan.
   - Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.

4. **Bauweise (§ 22 Ba

[['### Maß der baulichen Nutzung: \n1.2 Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)\n1.2.1 Grundflächenzahl (§ 19 BauNVO)\n- siehe Einschriebe im Lageplan\n\n1.2.2 Geschossflächenzahl (§ 20 BauNVO)\n- siehe Einschriebe im Lageplan\n\n1.2.3 Höhe der baulichen Anlagen (§ 18 BauNVO)\n- siehe Einschriebe im Lageplan\n- Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.\n\n### Referenzen: Seite 3'],
 ['### Maß der baulichen Nutzung: \n1. **Grundflächenzahl (§ 19 BauNVO)**\n   - Siehe Einschreibung im Lageplan.\n\n2. **Geschossflächenzahl (§ 20 BauNVO)**\n   - Siehe Einschreibung im Lageplan.\n\n3. **Höhe der baulichen Anlagen (§ 18 BauNVO)**\n   - Siehe Einschreibung im Lageplan.\n   - Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.\n\n4. **Bauweise (§ 22 BauNVO)**\n   - Siehe 

### D) Bauweise, überbaubare Grundstücksflächen

1. text_prompts
2. img_prompts

In [12]:
%store -r msgbaunvo_bg msgf11pdf
context_bg = parser.text2prompts([msgbaunvo_bg])

In [13]:
# D1) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store
instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".  Achte auf die Stuktur.'

def run():
    return ava.similaritySearchWithContext(instruction, msgf11pdf, context_bg)
runner.consistency_check(run)

['### Bauweise, überbaubare Grundstücksflächen

#### § 22 Bauweise
(1) Im Bebauungsplan kann die Bauweise als offene oder geschlossene Bauweise festgesetzt werden.  
(2) In der offenen Bauweise werden die Gebäude mit seitlichem Grenzabstand als Einzelhäuser, Doppelhäuser oder Hausgruppen errichtet.  
(3) In der geschlossenen Bauweise werden die Gebäude ohne seitlichen Grenzabstand errichtet.

#### § 23 Überbaubare Grundstücksfläche
(1) Die überbaubaren Grundstücksflächen können durch die Festsetzung von Baugrenzen, Baulinien oder Bebauungstiefen bestimmt werden.  
(2) Ist eine Baulinie festgesetzt, so muss auf dieser Linie gebaut werden.  
(3) Ist eine Baugrenze festgesetzt, so dürfen Gebäude und Gebäudeteile diese nicht überschreiten.

### Schriftlicher Teil Bebauungsplan "Gewerbegebiet Himmelreich"
#### Stadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis, 19.06.2000

#### 1.3 Bauweise (§ 22 BauNVO)
- siehe Einschriebe im Lageplan -  
a = abweichende Bauweise. Es gilt die offene 

[['### Bauweise, überbaubare Grundstücksflächen\n\n#### § 22 Bauweise\n(1) Im Bebauungsplan kann die Bauweise als offene oder geschlossene Bauweise festgesetzt werden.  \n(2) In der offenen Bauweise werden die Gebäude mit seitlichem Grenzabstand als Einzelhäuser, Doppelhäuser oder Hausgruppen errichtet.  \n(3) In der geschlossenen Bauweise werden die Gebäude ohne seitlichen Grenzabstand errichtet.\n\n#### § 23 Überbaubare Grundstücksfläche\n(1) Die überbaubaren Grundstücksflächen können durch die Festsetzung von Baugrenzen, Baulinien oder Bebauungstiefen bestimmt werden.  \n(2) Ist eine Baulinie festgesetzt, so muss auf dieser Linie gebaut werden.  \n(3) Ist eine Baugrenze festgesetzt, so dürfen Gebäude und Gebäudeteile diese nicht überschreiten.\n\n### Schriftlicher Teil Bebauungsplan "Gewerbegebiet Himmelreich"\n#### Stadt Laichingen, Gemarkung Feldstetten, Alb-Donau-Kreis, 19.06.2000\n\n#### 1.3 Bauweise (§ 22 BauNVO)\n- siehe Einschriebe im Lageplan -  \na = abweichende Bauweise. E

In [14]:
# D3) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'

async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_bg], img_type="pdf")
await runner.async_consistency_check(run)

['### Bauweise, überbaubare Grundstücksflächen:
1. **Bauweise (§ 22 BauNVO)**
   - Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.
   - Siehe Einschreibung im Lageplan.

2. **Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)**
   - Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in der Planzeichnung bestimmt.
   - Garagen im Sinne von § 12 BauNVO und Nebenanlagen im Sinne des § 14 BauNVO sowie Anlagen der Fremdenwerbung und bauliche Anlagen, die nach Landesrecht in den Abstandsflächen zulässig sind oder zugelassen werden können, sind nur innerhalb der überbaubaren Grundstücksflächen zulässig.

### Referenzen:
- Seite 4']
#############################################
['### Bauweise, überbaubare Grundstücksflächen:
1. **Bauweise (§ 22 BauNVO)**
   - Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.

2. **Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)**
   - Die überbaubare Grundstücksfläche wird durch die Fest

[['### Bauweise, überbaubare Grundstücksflächen:\n1. **Bauweise (§ 22 BauNVO)**\n   - Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.\n   - Siehe Einschreibung im Lageplan.\n\n2. **Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)**\n   - Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in der Planzeichnung bestimmt.\n   - Garagen im Sinne von § 12 BauNVO und Nebenanlagen im Sinne des § 14 BauNVO sowie Anlagen der Fremdenwerbung und bauliche Anlagen, die nach Landesrecht in den Abstandsflächen zulässig sind oder zugelassen werden können, sind nur innerhalb der überbaubaren Grundstücksflächen zulässig.\n\n### Referenzen:\n- Seite 4'],
 ['### Bauweise, überbaubare Grundstücksflächen:\n1. **Bauweise (§ 22 BauNVO)**\n   - Es gilt die offene Bauweise, jedoch sind die Gebäudelängen nicht begrenzt.\n\n2. **Überbaubare Grundstücksfläche (§ 9 (1) 2 BauGB)**\n   - Die überbaubare Grundstücksfläche wird durch die Festsetzung von Baugrenzen in de

### E) Alle Themen in einem Prompt

1. text_prompts (TODO)
2. img_prompts

In [15]:
%store -r msgbaunvo msgf11pdf
context_all = parser.text2prompts([msgbaunvo])

In [16]:
# E1) Textinformationen: Alle Themen – WITH CONTEXT – IMG_PROMPTS
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Achte auf die Stuktur.'

async def run():
    return await ava.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_all], img_type="pdf")
results = await runner.async_consistency_check(run)
msg232_f11_tt = results[0]
%store msg232_f11_tt

['### Art der baulichen Nutzung

1. **Planungsrechtliche Festsetzungen (§ 9 (1) und (2) BauGB und BauNVO)**
   - **1.1 Art der baulichen Nutzung (§§ 1 - 15 BauNVO)**
     - **1.1.1 Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)**
       - **1.1.1.1 Zulässig sind / zulässig ist:**
         - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
         - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
       - **1.1.1.2 Nicht zulässig sind / nicht zulässig ist:**
         - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
         - Ei

In [17]:
# E2) Textinformationen: Alle Themen – WITHOUT CONTEXT – IMG_PROMPTS
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

async def run():
    return await ava.extractTextFromImages(instruction, [pdf_path], img_type="pdf")
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan "Gewerbegebiet Himmelreich":

### Art der baulichen Nutzung
1. **Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)**
   - **Zulässig sind / zulässig ist:**
     - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.
     - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.
   - **Nicht zulässig sind / nicht zulässig ist:**
     - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.
     - Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nic

[['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan "Gewerbegebiet Himmelreich":\n\n### Art der baulichen Nutzung\n1. **Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)**\n   - **Zulässig sind / zulässig ist:**\n     - Gewerbebetriebe, die das Wohnen nicht wesentlich stören, wobei bei Einzelhandelsbetrieben (gem. § 1 (5) BauNVO i.V.m. § 1 (9) BauNVO) nur die nicht innenstadtrelevanten Sortimente Kfz, Motorräder, Mopeds, Fahrräder, Kfz-Zubehör, Rasenmäher, Landmaschinen, Fahrrad- und Motorradzubehör, Brennstoffe und Mineralölerzeugnisse zulässig sind.\n     - Der Verkauf von auf dem Grundstück produzierten Waren auf einer untergeordneten Fläche ist zulässig.\n   - **Nicht zulässig sind / nicht zulässig ist:**\n     - Nutzungen gemäß § 8 (3) Nr. 3 BauNVO (Vergnügungsstätten) sind gemäß § 1 (6) BauNVO nicht Bestandteil des Bebauungsplans und damit nicht zulässig.\n     - Einzelhandelsbetriebe mit innenstadtrelevanten und best